In [1]:
import os 
import sys
from importlib import reload

# Import Pytorch & Huggingface ==========================
#set cache dir before imports 
from configuration import huggingface_path as cache_dir
#cache_dir = cache_path_here
os.environ['HF_HOME'] = f"{cache_dir}/huggingface"
os.environ['TRANSFORMERS_CACHE']= f"{cache_dir}/huggingface/hub"
os.environ['TORCH_HOME']= f"{cache_dir}/torch"
os.environ['XDG_CACHE_HOME']= cache_dir

#imports 
import torch
from torch.cuda import memory_allocated, empty_cache
from datasets import load_dataset
from torch.utils.data import DataLoader

# Import local functions ==================================
import activation_extractor
from activation_extractor import *
reload(activation_extractor)

/orfeo/cephfs/scratch/area/evillegas/glm/dgxtorch/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


<module 'activation_extractor' from '/orfeo/cephfs/scratch/area/evillegas/glm/activation-extractor/src/activation_extractor/__init__.py'>

In [2]:
from activation_extractor.extractors import intermediateExtractorBase 
reload(intermediateExtractorBase)
from activation_extractor.inferencers import inferencerBase
reload(inferencerBase)
from activation_extractor.model_functions import load_models, tokenize_funs, inference_funs, default_hooked_layers
reload(load_models)
reload(tokenize_funs)
reload(inference_funs)
reload(default_hooked_layers)

<module 'activation_extractor.model_functions.default_hooked_layers' from '/orfeo/cephfs/scratch/area/evillegas/glm/activation-extractor/src/activation_extractor/model_functions/default_hooked_layers.py'>

In [3]:
#install editable version of this repository
#!python -m pip install -e .

# Proteins/DNA

In [4]:
#load data
sequences = ["AAAAAAAAAAA", "HHHHHHHHHHHHHH"]

In [13]:
model_name = "facebook/esm2_t6_8M_UR50D"
inferencer = activation_extractor.Inferencer(model_name, device='cuda', half=False)

In [14]:
#intermediate activation extractor
layers_to_hook = activation_extractor.get_layers_to_hook(inferencer.model,inferencer.model_type)
extractor = activation_extractor.IntermediateExtractor(inferencer.model, layers_to_hook)
extractor.register_hooks()

#inference
processed = inferencer.process(sequences) #tokenize
outputs = inferencer.inference(processed)

#extractor outputs
#extractor.save_outputs('results/embeddings/test')
extractor.clear_all_hooks()

In [15]:
extractor.get_outputs()

{'esm.embeddings': tensor([[[-0.1720, -0.2421, -0.1315,  ...,  0.1121, -0.0918, -0.0569],
          [-0.0587, -0.0105,  0.0076,  ...,  0.0129, -0.0387, -0.1703],
          [-0.0587, -0.0105,  0.0076,  ...,  0.0129, -0.0387, -0.1703],
          ...,
          [-0.0000, -0.0000, -0.0000,  ..., -0.0000, -0.0000,  0.0000],
          [-0.0000, -0.0000, -0.0000,  ..., -0.0000, -0.0000,  0.0000],
          [-0.0000, -0.0000, -0.0000,  ..., -0.0000, -0.0000,  0.0000]],
 
         [[-0.1720, -0.2421, -0.1315,  ...,  0.1121, -0.0918, -0.0569],
          [-0.1050, -0.0207,  0.0303,  ..., -0.0258,  0.1157,  0.1383],
          [-0.1050, -0.0207,  0.0303,  ..., -0.0258,  0.1157,  0.1383],
          ...,
          [-0.1050, -0.0207,  0.0303,  ..., -0.0258,  0.1157,  0.1383],
          [-0.1050, -0.0207,  0.0303,  ..., -0.0258,  0.1157,  0.1383],
          [ 0.0643, -0.2711, -0.1030,  ..., -0.1941, -0.3021, -0.0014]]],
        device='cuda:0', grad_fn=<MulBackward0>),
 'esm.encoder.layer.0.output': te

# Images 

In [3]:
from PIL import Image
import requests

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

In [4]:
model_name = "timm/vgg16.tv_in1k"
inferencer = activation_extractor.Inferencer(model_name, device='cuda', half=False)

In [5]:
#intermediate activation extractor
layers_to_hook = default_hooked_layers.get_layers_to_hook(inferencer.model,inferencer.model_type)
extractor = intermediateExtractorBase.IntermediateExtractorBase(inferencer.model, layers_to_hook)
extractor.register_hooks()

#inference
processed = inferencer.process(image)
outputs = inferencer.inference(processed)

#extractor outputs
#extractor.save_outputs('results/embeddings/test')
extractor.clear_all_hooks()

In [6]:
extractor.get_outputs()

{'features.0': tensor([[[[2.5090, 1.0410, 1.0889,  ..., 0.8414, 0.7990, 0.0000],
           [3.2924, 0.9930, 0.9942,  ..., 1.0386, 0.9353, 0.0000],
           [3.3463, 0.9217, 0.8648,  ..., 1.0222, 0.9266, 0.0000],
           ...,
           [3.2679, 1.3011, 1.2664,  ..., 1.3564, 1.5469, 0.0000],
           [3.3174, 1.4716, 1.3920,  ..., 1.3474, 1.4371, 0.0000],
           [2.5755, 1.3999, 1.3387,  ..., 1.2332, 1.2578, 0.0000]],
 
          [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 2.3871],
           [0.0000, 0.3414, 0.3781,  ..., 0.4543, 0.3218, 3.7593],
           [0.0000, 0.3860, 0.2605,  ..., 0.4514, 0.3712, 4.0382],
           ...,
           [1.6528, 0.3176, 0.3506,  ..., 0.3592, 0.3019, 0.0000],
           [1.7996, 0.5880, 0.2601,  ..., 0.2439, 0.3027, 0.0000],
           [0.6340, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
 
          [[1.6709, 1.8936, 2.0400,  ..., 3.3777, 3.2689, 5.0077],
           [0.6761, 0.5608, 0.6890,  ..., 0.5908, 0.6574, 2.3777],
          

# Multimodal

In [19]:
from PIL import Image
import requests

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
text = ["a photo of a cat", "a photo of a dog"]

input_data = {"text":text,
             "image":image}

In [23]:
model_name = "openai/clip-vit-base-patch32"
inferencer = activation_extractor.Inferencer(model_name, device='cuda', half=False)

In [24]:
#intermediate activation extractor
layers_to_hook = default_hooked_layers.get_layers_to_hook(inferencer.model,inferencer.model_type)
extractor = intermediateExtractorBase.IntermediateExtractorBase(inferencer.model, layers_to_hook)
extractor.register_hooks()

#inference
processed = inferencer.process(input_data)
outputs = inferencer.inference(processed)

#extractor outputs
#extractor.save_outputs('results/embeddings/test')
extractor.clear_all_hooks()

In [25]:
extractor.get_outputs()

{'vision_model.embeddings': tensor([[[ 0.0157,  0.2836, -0.1435,  ...,  0.0176, -0.3890, -0.2827],
          [-0.0363, -0.0173, -0.0764,  ..., -0.1424, -0.4067, -0.5826],
          [-0.0629, -0.0032,  0.0320,  ..., -0.1494, -0.4059, -0.6116],
          ...,
          [-0.2810, -0.1870, -0.2222,  ..., -0.2507, -0.2175, -0.6715],
          [-0.2264, -0.1276, -0.3738,  ..., -0.2622, -0.1628, -0.5990],
          [-0.3358, -0.1170, -0.2031,  ..., -0.2388, -0.2045, -0.6593]]],
        device='cuda:0', grad_fn=<AddBackward0>),
 'vision_model.encoder.layers.0': (tensor([[[ 0.0714,  0.1635, -0.1355,  ...,  0.1418,  0.0949,  0.0236],
           [ 0.1329, -0.0728, -0.0854,  ...,  0.2067,  0.1030, -0.0342],
           [ 0.1133, -0.0808, -0.0419,  ...,  0.1583,  0.0953, -0.0451],
           ...,
           [-0.0174, -0.0332, -0.1135,  ..., -0.1081,  0.0496, -0.0299],
           [ 0.0218, -0.0245, -0.1617,  ..., -0.1694,  0.0417, -0.0145],
           [-0.0742, -0.0365, -0.1074,  ..., -0.1154,  0.052